In [5]:
import csv
import datetime
import numpy as np
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from requests import get
from preprocessing import uniformName

ImportError: cannot import name 'uniformName'

In [2]:
df = pd.read_excel('data/base_data.xlsx')
playersSet = set(df['Winner']).union(set(df['Loser']))
players = {n : [] for n in playersSet}

In [3]:
def write_csv(players):
    with open('data/playersdata.csv', 'w', newline='', encoding='utf-8-sig') as file :
        writer = csv.writer(file)
        writer.writerow(['Player', 'BirthDate', 'Hand', 'BackHand'])
        for p in players.values():
            writer.writerow(p)

In [4]:
def static_scraping(players):
    url = 'http://tennisabstract.com/reports/atp_elo_ratings.html'
    r =  get(url)
    page = BeautifulSoup(r.content)
    table = page.find(id='reportable')
    allTr = table.find_all('tr')

    for tr in allTr[1:] :
        tds = tr.find_all('td')
        toWrite = [tds[i].text for i in (0, 1, 3, 5, 6, 7, 9, 10, 11)]

        toWrite[1] = uniformName(toWrite[1])

        playerLink = tr.a['href']
        raw = get(playerLink)
        playerPage = BeautifulSoup(raw.content)
        s = playerPage.find('script', language='JavaScript').text
        i = s.find('var dob =')
        dob = datetime.date(int(s[i+10:i+14]), int(s[i+14:i+16]), int(s[i+16:i+18]))
        i = s.find('var hand =')
        hand = s[i+12]
        if hand not in {'L', 'R'}:
            hand = None
        i = s.find('var backhand =')
        backhand = s[i+16]
        if backhand not in {'1', '2'}:
            backhand = None

        toWrite += [dob, hand, backhand]
        players[toWrite[1]] = toWrite

In [5]:
def dynamic_scraping(playerNames):
    driver = webdriver.Chrome()
    driver.get("https://www.atptour.com/en/players")
    items = list(playerNames.items())
    for w, values in items:
        surname = w.split()[0]
        search = driver.find_element_by_name("playerInput")
        search.clear()
        search.click()
        search.send_keys(surname+ " ")
        ul = driver.find_element_by_id("playerDropdown")
        sleep(8)
        
        try:   
            playersList = ul.find_elements_by_partial_link_text(" " + surname)
            for p in playersList:
                pName = p.text.replace('-', ' ').split()
                initials = w.split()[1].split('.')[:-1]
                print(pName , ' --> ' , initials)
                found = True
                for i in range(len(initials)):
                    found = found and initials[i] == pName[i][0]
                if found:
                    player = p
                    break
                    
            if not found:    
                raise Exception(w + ' not found')
                
            print(w + ": " + player.text)
            player.click()
            sleep(8)
                 
            playerInfo = driver.find_element_by_css_selector("div.player-profile-hero-table")
            pBD = playerInfo.find_element_by_css_selector("span.table-birthday").text
            pBD = pBD[1:-1].replace('.', '-')

            play = driver.find_element_by_xpath('//*[@id="playerProfileHero"]/div[2]/div[2]/div/table/tbody/tr[2]/td[3]/div/div[2]').text
            hand = play[0]
            if hand not in {'L', 'R'}:
                        hand = None
            index = play.find('-Handed Backhand')
            backHand = play[index-3 : index]
            if backHand == "One" : 
                backHand = 1
            elif backHand == "Two":
                backHand = 2
            else:
                backHand = None
        except Exception as e:
            print(e)
            pBD = None
            hand = None
            backHand = None
            
        
        playerNames[w] = [w, pBD, hand, backHand]
    driver.close()

In [6]:
#print("Static Scraping started")
#static_scraping(players)
print("Dynamic Scraping started")
dynamic_scraping(players)
print("Writing...")
write_csv(players)
print("Done")

Dynamic Scraping started
['Alexander', 'Sarkissian']  -->  ['A']
Sarkissian A.: Alexander Sarkissian
['Jerzy', 'Janowicz']  -->  ['J']
Janowicz J.: Jerzy Janowicz
['Cary', 'Rubin']  -->  ['N']
['Ed', 'Rubin']  -->  ['N']
['Edouard', 'Georges', 'Rubinoff']  -->  ['N']
['Eric', 'Rubin']  -->  ['N']
['James', 'Rubin']  -->  ['N']
['Kevin', 'Rubink']  -->  ['N']
['Michael', 'Rubin']  -->  ['N']
['Noah', 'Rubin']  -->  ['N']
Rubin N.: Noah Rubin
['Giovanni', 'Galvani']  -->  ['S']
['Stefano', 'Galvani']  -->  ['S']
Galvani S.: Stefano Galvani
['Bernardo', 'Mello', 'Madeira', 'Coelho']  -->  ['R']
['Evelyn', 'Mellor']  -->  ['R']
['Henrique', 'Mello']  -->  ['R']
['Ricardo', 'Mello']  -->  ['R']
Mello R.: Ricardo Mello
['Diego', 'Junqueira']  -->  ['D']
Junqueira D.: Diego Junqueira
['Prateek', 'Bhambri']  -->  ['Y']
['Yuki', 'Bhambri']  -->  ['Y']
Bhambri Y.: Yuki Bhambri
['Marc', 'Andrea', 'Huesler']  -->  ['M', 'A']
Huesler M.A.: Marc-Andrea Huesler
['Anil', 'Yuksel']  -->  ['A']
Yuksel A

['Brandon', 'Walters']  -->  ['L']
['Christian', 'Walter']  -->  ['L']
['Christian', 'Walter']  -->  ['L']
['Florent', 'Walter']  -->  ['L']
['Jesse', 'Walter']  -->  ['L']
['Jesse', 'Walter']  -->  ['L']
['Ludovic', 'Walter']  -->  ['L']
Walter L.: Ludovic Walter
string index out of range
['Przemysaw', 'Kwiatkowski']  -->  ['T', 'S']
['Thai', 'Son', 'Kwiatkowski']  -->  ['T', 'S']
Kwiatkowski T.S.: Thai-Son Kwiatkowski
['Roman', 'Borvanov']  -->  ['R']
Borvanov R.: Roman Borvanov
['O.', 'Malisse']  -->  ['X']
['Xavier', 'Malisse']  -->  ['X']
Malisse X.: Xavier Malisse
['Lukasz', 'Kubot']  -->  ['L']
Kubot L.: Lukasz Kubot
['Karim', 'Hossam']  -->  ['K']
Hossam K.: Karim Hossam
['Mikhail', 'Biryukov']  -->  ['M']
Biryukov M.: Mikhail Biryukov
string index out of range
['Alejandro', 'Falla']  -->  ['A']
Falla A.: Alejandro Falla
['Jose', 'Hernandez', 'Fernandez']  -->  ['J']
Hernandez-Fernandez J.: Jose Hernandez-Fernandez
['Sean', 'Alan', 'Kolar']  -->  ['Z']
['Igor', 'Kolaric']  --> 

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.117)

Message: stale element reference: element is not

['Luis', 'David', 'Martinez']  -->  ['P']
['Denilson', 'Martinez']  -->  ['P']
['Diego', 'Martinez', 'del', 'Rio']  -->  ['P']
['Jose', 'Diego', 'Martinez']  -->  ['P']
['Diego', 'Martinez']  -->  ['P']
['Dimitrio', 'Martinez']  -->  ['P']
['Douglas', 'Martinez']  -->  ['P']
['Eduardo', 'Martinez']  -->  ['P']
['Erick', 'Eduardo', 'Martinez', 'Torres']  -->  ['P']
['Emilio', 'Martinez', 'Del', 'Rey']  -->  ['P']
['Erick', 'Martinez']  -->  ['P']
['Ernesto', 'Sanz', 'Martinez']  -->  ['P']
['Erwin', 'Francisco', 'Lopez', 'Martinez']  -->  ['P']
['Esdras', 'Martinez']  -->  ['P']
['Jose', 'Eugenio', 'Martinez', 'Alcalde']  -->  ['P']
['Felipe', 'Martinez', 'Sarrasague']  -->  ['P']
['Fermin', 'Peralta', 'Martinez']  -->  ['P']
['Francisco', 'Martinez', 'Lidon']  -->  ['P']
['Geoff', 'Martinez']  -->  ['P']
['Gerardo', 'Martinez']  -->  ['P']
['Giovanni', 'Martinez']  -->  ['P']
['Gonzalo', 'Andres', 'Martinez']  -->  ['P']
['Ignacio', 'Martinez']  -->  ['P']
Martinez P. not found
['Facun

string index out of range
['Rainer', 'Schuettler']  -->  ['R']
Schuettler R.: Rainer Schuettler
['Andrei', 'Youzhny']  -->  ['M']
['Mikhail', 'Youzhny']  -->  ['M']
Youzhny M.: Mikhail Youzhny
['Antonio', 'Caruso']  -->  ['S']
['Salvatore', 'Caruso']  -->  ['S']
Caruso S.: Salvatore Caruso
['Mario', 'Mansilla', 'Diez']  -->  ['S']
['Raymond', 'Diez']  -->  ['S']
['Steven', 'Diez']  -->  ['S']
Diez S.: Steven Diez
['Carlos', 'Rios']  -->  ['F']
['Christian', 'De', 'Los', 'Rios']  -->  ['F']
['Elkin', 'Rios']  -->  ['F']
['Ernesto', 'Rios']  -->  ['F']
['Felipe', 'Rios']  -->  ['F']
Rios F.: Felipe Rios
['Daniel', 'Masur']  -->  ['D']
Masur D.: Daniel Masur
['Sandro', 'Ehrat']  -->  ['S']
Ehrat S.: Sandro Ehrat
['Louis', 'Wessels']  -->  ['L']
Wessels L.: Louis Wessels
['Adolfo', 'Gomez']  -->  ['L']
['Adolfo', 'Gomez', 'Pinter']  -->  ['L']
['Federico', 'Agustin', 'Gomez']  -->  ['L']
['Alejandro', 'Jose', 'Gomez', 'Estrada']  -->  ['L']
['Alejandro', 'Gomez']  -->  ['L']
['Alejandro', 

...

string index out of range
['Germain', 'Gigounon']  -->  ['G']
Gigounon G.: Germain Gigounon
['Adam', 'Becker']  -->  ['K']
['Alec', 'Beckley']  -->  ['K']
['Andreas', 'Beck']  -->  ['K']
['Bastian', 'Beck']  -->  ['K']
['Benjamin', 'Becker']  -->  ['K']
['Boris', 'Becker']  -->  ['K']
['C.', 'Beck']  -->  ['K']
['Carlos', 'Becke']  -->  ['K']
['Chad', 'Gary', 'Becker']  -->  ['K']
['Charles', 'Beckman']  -->  ['K']
['David', 'Beckman']  -->  ['K']
['Gerhardt', 'Marius', 'Becker']  -->  ['K']
['Harvey', 'Becker']  -->  ['K']
['Jaromir', 'Becka']  -->  ['K']
['Jean', 'Becker']  -->  ['K']
['Jerome', 'Becker']  -->  ['K']
['Jim', 'Beckwith']  -->  ['K']
['Juan', 'Pablo', 'Becker']  -->  ['K']
['Kai', 'Becker']  -->  ['K']
Beck K.: Kai Becker
Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.table-birthday"}
  (Session info: chrome=79.0.3945.117)

['Josselin', 'Ouanna']  -->  ['J']
Ouanna J.: Josselin Ouanna
['Nick', 'Kyrgios']  -->  ['N']
Kyrgio

['Chris', 'Herbert']  -->  ['P']
['Pierre', 'Hugues', 'Herbert']  -->  ['P']
Herbert P.H: Pierre-Hugues Herbert
['Pablo', 'Carreno', 'Busta']  -->  []
Carreno Busta P.: Pablo Carreno Busta
['Adam', 'Becker']  -->  ['B']
['Benjamin', 'Becker']  -->  ['B']
Becker B.: Benjamin Becker
['Luis', 'Enrique', 'Barrientos']  -->  ['N']
['Felipe', 'Barrientos']  -->  ['N']
['Felix', 'Barrientos']  -->  ['N']
['Hipolito', 'Barrientos']  -->  ['N']
['Nicolas', 'Barrientos']  -->  ['N']
Barrientos N.: Nicolas Barrientos
['Chia', 'Cheng', 'Yang']  -->  ['T', 'H']
['Shao', 'Chi', 'Yang']  -->  ['T', 'H']
['Dae', 'Yang', 'Kweon']  -->  ['T', 'H']
['Zi', 'Dong', 'Yang']  -->  ['T', 'H']
['Eui', 'Yeol', 'Yang']  -->  ['T', 'H']
['Yi', 'Fan', 'Yang']  -->  ['T', 'H']
['Wei', 'Guang', 'Yang']  -->  ['T', 'H']
['Hao', 'Yang', 'Chen']  -->  ['T', 'H']
['Tsung', 'Hua', 'Yang']  -->  ['T', 'H']
Yang T.H.: Tsung-Hua Yang
Message: stale element reference: element is not attached to the page document
  (Session i

['Mousa', 'Shanan', 'Zayed']  -->  ['M']
Zayed M. S.: Mousa Shanan Zayed
string index out of range
['Adan', 'Freire', 'Da', 'Silva']  -->  ['F', 'F']
['Alexandre', 'Guilherme', 'Silva', 'Nunes']  -->  ['F', 'F']
['Allan', 'Da', 'Silva', 'Souza', 'Ferreira']  -->  ['F', 'F']
['Anderson', 'Silva']  -->  ['F', 'F']
['Andre', 'Luiz', 'Borghi', 'Silvares']  -->  ['F', 'F']
['Andre', 'Silva', 'Ferreira']  -->  ['F', 'F']
['Andre', 'Silva']  -->  ['F', 'F']
['Jarisson', 'Aquino', 'Silva']  -->  ['F', 'F']
['B.', 'Da', 'Silva']  -->  ['F', 'F']
['Braulino', 'Silva', 'Jr.']  -->  ['F', 'F']
['Bruno', 'Silva']  -->  ['F', 'F']
['Caio', 'Silva']  -->  ['F', 'F']
['Juan', 'Carlos', 'Silva']  -->  ['F', 'F']
['Julio', 'Cesar', 'Chaves', 'Silva']  -->  ['F', 'F']
['Paulo', 'Cesar', 'Silva', 'Dos', 'Santos']  -->  ['F', 'F']
['Christopher', 'Ross', 'Da', 'Silva']  -->  ['F', 'F']
['Gabriel', 'Ciro', 'Da', 'Silva']  -->  ['F', 'F']
['Daniel', 'Dutra', 'da', 'Silva']  -->  ['F', 'F']
['Jadielson', 'Sil

['Michael', 'Yani']  -->  ['M']
Yani M.: Michael Yani
['Marco', 'Viola']  -->  ['M']
Viola M.: Marco Viola
string index out of range
['Viktor', 'Galovic']  -->  ['V']
Galovic V.: Viktor Galovic
['Niels', 'Desein']  -->  ['N']
Desein N.: Niels Desein
['Jun', 'tong', 'Gao']  -->  ['X']
['Lei', 'Gao']  -->  ['X']
['Peng', 'Gao']  -->  ['X']
['Qun', 'Gao']  -->  ['X']
['Sergio', 'Gaoni']  -->  ['X']
['Wan', 'Gao']  -->  ['X']
['Xin', 'Gao']  -->  ['X']
Gao X.: Xin Gao
['Gabriel', 'A.', 'Penino', 'De', 'Souza']  -->  []
De Paula F.: Gabriel A. Penino De Souza
['Andrei', 'Medvedev']  -->  ['D']
['Daniil', 'Medvedev']  -->  ['D']
Medvedev D.: Daniil Medvedev
['Chris', 'Kwon']  -->  ['S', 'W']
['Oh', 'Hee', 'Kwon']  -->  ['S', 'W']
['Yong', 'Ho', 'Kwon']  -->  ['S', 'W']
['Hyon', 'Te', 'Kwon']  -->  ['S', 'W']
['Hyung', 'Tae', 'Kwon']  -->  ['S', 'W']
['Michael', 'Kwong']  -->  ['S', 'W']
['Soonwoo', 'Kwon']  -->  ['S', 'W']
['Tong', 'Su', 'Kwon']  -->  ['S', 'W']
['Young', 'Min', 'Kwon']  -->

['Grega', 'Zemlja']  -->  ['G']
Zemlja G.: Grega Zemlja
['Marko', 'Sekulic']  -->  ['M']
Sekulic M.: Marko Sekulic
string index out of range
['Vincent', 'Millot']  -->  ['V']
Millot V.: Vincent Millot
['Adolfo', 'Daniel', 'Vallejo']  -->  ['M']
['Alejandro', 'Daniel', 'Castillo', 'Gamarra']  -->  ['M']
['Alvaro', 'Daniel', 'Riveros']  -->  ['M']
['Arman', 'Danielyan']  -->  ['M']
['Bohus', 'Danielcik']  -->  ['M']
['Bruce', 'Daniels']  -->  ['M']
['Christian', 'Daniel', 'Bianculli']  -->  ['M']
['Christofer', 'Daniel', 'Goncalves']  -->  ['M']
['Joao', 'Daniel', 'Almeida']  -->  ['M']
['Pedro', 'Daniel', 'Gutierrez', 'Arreaga']  -->  ['M']
['Paul', 'Daniel', 'Gijon', 'Arruabarrena']  -->  ['M']
['Emilio', 'Daniel', 'Baez', 'Britez']  -->  ['M']
['Jorge', 'Daniel', 'Barcenas', 'Porcell']  -->  ['M']
['Jose', 'Daniel', 'Bendeck']  -->  ['M']
Daniel M. not found
['Aeron', 'Martinovic']  -->  ['A']
Martin A.: Aeron Martinovic
['Diego', 'Schwartzman']  -->  ['D']
Schwartzman D.: Diego Schwa

## Merging Data

In [10]:
df_ext = pd.read_excel('data/base_data.xlsx')
scraped_data = pd.read_csv('data/playersdata.csv', encoding='utf-8-sig')

In [11]:
def fill_values(row, scraped): 
    wBirthDate = None
    wHand = None
    wBackhand = None

    lBirthDate = None
    lHand = None
    lBackhand = None
        
    wFound = False
    lFound = False
    for i, r in scraped.iterrows() :
        if(row['Winner']  == r['Player']) :
            wFound = True
            wBirthDate = r['BirthDate']
            wHand = r['Hand']
            wBackhand = r['BackHand']
            
        elif (row['Loser']  == r['Player']) :
            lFound = True
            lBirthDate = r['BirthDate']
            lHand = r['Hand']
            lBackhand = r['BackHand']

        elif wFound and lFound :
            break
        
    print(row.name, row['Winner'], row['Loser'])
    return wBirthDate, wHand, wBackhand, lBirthDate, lHand, lBackhand

In [12]:
def join_datasets(base, scraped):
    result = base.apply(lambda row: fill_values(row, scraped), axis=1, result_type='expand')
            
    base = base.assign(WBD=result[0], WHand=result[1], WBHand=result[2],
                       LBD=result[3], LHand=result[4], LBHand=result[5])
    
    base['Date'] = pd.to_datetime(base['Date'])
    return base.sort_values(by='Date')

def write_dataset(base):
    os.chdir('data')
    with open('dataset.csv', 'w', newline='', encoding='utf-8-sig') as file :
        writer = csv.writer(file)
        writer.writerow([c for c in base.columns])
        for i, r in df_ext.iterrows():
            writer.writerow(r)
    os.chdir('..')
            
def split_by_year(base):
    os.chdir('data')
    for year in base.Date.dt.year.unique():
        base[base.Date.dt.year == year].to_csv(str(year) + '.csv', 
                                               encoding='utf-8-sig',
                                               index=False)
    os.chdir('..')

In [14]:
df_ext = join_datasets(df_ext, scraped_data)
write_dataset(df_ext)
split_by_year(df_ext)

0 Lopez F. Petzschner P.
1 Istomin D. De Bakker T.
2 Berrer M. Sela D.
3 Kubot L. Brands D.
4 Mayer F. Tomic B.
5 Giraldo S. Simon G.
6 Dolgopolov O. Andreev I.
7 Roddick A. Matosevic M.
8 Kendrick R. Gremelmayr D.
9 Phau B. Bolelli S.
10 Nishikori K. Cilic M.
11 Robert S. Haider-Maurer A.
12 Dodig I. Daniel M.
13 Tipsarevic J. Schwank E.
14 Garcia-Lopez G. Kukushkin M.
15 Tsonga J.W. Ramirez-Hidalgo R.
16 Davydenko N. Fognini F.
17 Marchenko I. Zverev M.
18 Bubka S. Gimeno-Traver D.
19 Stepanek R. Kamke T.
20 Becker B. Verdasco F.
21 Anderson K. Luczak P.
22 Berankis R. Clement A.
23 Fish M. Mannarino A.
24 Ebden M. Millman J.
25 Baghdatis M. Serra F.
26 Soderling R. Harrison R.
27 Kudryavtsev A. Bhambri Y.
28 Gasquet R. Roger-Vasselin E.
29 Malisse X. Ilhan M.
30 Berdych T. Gil F.
31 Haase R. Dancevic F.
32 Falla A. Machado R.
33 Wawrinka S. Bopanna R.
34 Goffin D. Devvarman S.
35 Sugita Y. Brown D.
36 Kavcic B. Chardy J.
37 Chiudinelli M. El Amrani R.
38 Gulbis E. Hanescu V.
39 Gaba

316 Devvarman S. Huta Galung J.
317 Van Der Merwe I. Brown D.
318 Greul S. Raonic M.
319 Zampieri C. Massu N.
320 Fognini F. Ramos-Vinolas A.
321 Gonzalez M. Souza J.
322 Bellucci T. Machado R.
323 Marchenko I. Lajovic D.
324 Clement A. Kukushkin M.
325 Mayer F. Stepanek R.
326 Garcia-Lopez G. Lacko L.
327 Cilic M. Kunitsyn I.
328 Berrer M. Kavcic B.
329 Beck K. Soeda G.
330 Anderson K. Sela D.
331 Mannarino A. Gremelmayr D.
332 Dancevic F. Wolmarans F.
333 Robredo T. Mello R.
334 Giraldo S. Andreev I.
335 Starace P. Andujar P.
336 Chela J.I. Zampieri C.
337 Bogomolov A. Petzschner P.
338 Gasquet R. Clement A.
339 Ljubicic I. Brands D.
340 Dodig I. Marchenko I.
341 Van Der Merwe I. Greul S.
342 Devvarman S. De Voest R.
343 Mannarino A. Dancevic F.
344 Anderson K. Beck K.
345 Zeballos H. Nalbandian D.
346 Giraldo S. Chela J.I.
347 Starace P. Zeballos H.
348 Fognini F. Bellucci T.
349 Berrer M. Gasquet R.
350 Garcia-Lopez G. Bogomolov A.
351 Mayer F. Cilic M.
352 Dodig I. Ljubicic I.
353

623 Tipsarevic J. Dodig I.
624 Djokovic N. Berdych T.
625 Federer R. Gasquet R.
626 Ferrer D. Dolgopolov O.
627 Fish M. Falla A.
628 Del Potro J.M. Anderson K.
629 Tipsarevic J. Nishikori K.
630 Djokovic N. Federer R.
631 Ferrer D. Almagro N.
632 Del Potro J.M. Fish M.
633 Del Potro J.M. Tipsarevic J.
634 Tipsarevic J. Kamke T.
635 Young D. Starace P.
636 Sweeting R. Granollers M.
637 Hanescu V. Seppi A.
638 Devvarman S. Mannarino A.
639 Berrer M. Phau B.
640 Smyczek T. Marchenko I.
641 De Voest R. Riba P.
642 Schuettler R. Haase R.
643 Malisse X. Cipolla F.
644 Del Potro J.M. Stepanek R.
645 Zverev M. Ebden M.
646 Karlovic I. Matosevic M.
647 Berankis R. Bogomolov A.
648 Nieminen J. Istomin D.
649 Russell M. Anderson K.
650 Cuevas P. Andujar P.
651 Harrison R. Chardy J.
652 Mello R. Ramirez-Hidalgo R.
653 Raonic M. Ilhan M.
654 Giraldo S. Gimeno-Traver D.
655 Petzschner P. Brown D.
656 Serra F. Gabashvili T.
657 Becker B. De Bakker T.
658 Benneteau J. Lopez F.
659 Andreev I. Nishikori

933 Kavcic B. Kamke T.
934 Ramos A. Ramirez-Hidalgo R.
935 Hanescu V. Golubev A.
936 Granollers M. Brands D.
937 Haase R. Cipolla F.
938 Giraldo S. Bellucci T.
939 Vagnozzi S. Fognini F.
940 Paire B. Carreno-Busta P.
941 Garcia-Lopez G. Istomin D.
942 Roger-Vasselin E. Kuznetsov An.
943 Montanes A. Gabashvili T.
944 Andujar P. Granollers-Pujol G.
945 Anderson K. Cuevas P.
946 Monaco J. Dimitrov G.
947 Gimeno-Traver D. Starace P.
948 Dodig I. Millot V.
949 Nishikori K. Riba P.
950 Greul S. Kendrick R.
951 Gasquet R. Chela J.I.
952 Ferrero J.C. Malisse X.
953 Berlocq C. Nieminen J.
954 Lopez F. Kukushkin M.
955 Raonic M. Stepanek R.
956 Davydenko N. Dolgopolov O.
957 Almagro N. Andujar P.
958 Monfils G. Haase R.
959 Nishikori K. Machado R.
960 Giraldo S. Ramos A.
961 Ferrero J.C. Zverev M.
962 Vagnozzi S. Monaco J.
963 Montanes A. Paire B.
964 Ferrer D. Berlocq C.
965 Hanescu V. Anderson K.
966 Nadal R. Gimeno-Traver D.
967 Lopez F. Garcia-Lopez G.
968 Gasquet R. Kavcic B.
969 Melzer J. 

1234 Falla A. Starace P.
1235 Fish M. Mello R.
1236 Andreev I. Serra F.
1237 Federer R. Lopez F.
1238 Monfils G. Phau B.
1239 Tipsarevic J. Dabul B.
1240 Davydenko N. Gremelmayr D.
1241 Darcis S. Llodra M.
1242 Berlocq C. Tomic B.
1243 Bolelli S. Dancevic F.
1244 Petzschner P. Zverev M.
1245 Kamke T. Rochus O.
1246 Schoorel T. Gonzalez M.
1247 Gasquet R. Stepanek R.
1248 Hanescu V. Paire B.
1249 Mayer L. Brown D.
1250 Baghdatis M. Gil F.
1251 Granollers M. Bogomolov A.
1252 Veic A. Cuevas P.
1253 Berrer M. Raonic M.
1254 Dolgopolov O. Schuettler R.
1255 Ljubicic I. Devvarman S.
1256 Murray A. Prodon E.
1257 Haider-Maurer A. Sweeting R.
1258 Mayer F. Kunitsyn I.
1259 Clement A. Volandri F.
1260 Kubot L. Almagro N.
1261 Chela J.I. Smyczek T.
1262 Haase R. Gimeno-Traver D.
1263 Querrey S. Kohlschreiber P.
1264 Melzer J. Beck A.
1265 Andujar P. Giraldo S.
1266 Nadal R. Isner J.
1267 Soderling R. Harrison R.
1268 Malisse X. Tursunov D.
1269 Anderson K. Mahut N.
1270 Verdasco F. Monaco J.
12

...

22406 Kudla D. Monfils G.
22407 Berrettini M. Khachanov K.
22408 Brown D. Zverev A.
22409 Auger-Aliassime F. Simon G.
22410 Gasquet R. Kukushkin M.
22411 Coric B. Garin C.
22412 Mannarino A. Goffin D.
22413 Thompson J. De Minaur A.
22414 Gasquet R. Jarry N.
22415 Berrettini M. Kudla D.
22416 Struff J.L. Pouille L.
22417 Auger-Aliassime F. Brown D.
22418 Raonic M. Fucsovics M.
22419 Berrettini M. Struff J.L.
22420 Auger-Aliassime F. Raonic M.
22421 Thompson J. Gasquet R.
22422 Mannarino A. Coric B.
22423 Mannarino A. Thompson J.
22424 Berrettini M. Auger-Aliassime F.
22425 Albot R. Ebden M.
22426 Johnson S. Kohlschreiber P.
22427 Herbert P.H. Monfils G.
22428 Sousa J. Hurkacz H.
22429 Khachanov K. Kecmanovic M.
22430 Zverev A. Haase R.
22431 Medvedev D. Verdasco F.
22432 Schwartzman D. Bublik A.
22433 Cilic M. Garin C.
22434 Mahut N. Tiafoe F.
22435 Anderson K. Norrie C.
22436 Simon G. Ward J.
22437 Gasquet R. Gojowczyk P.
22438 Goffin D. Pella G.
22439 Struff J.L. Djere L.
22440 Bautis

22691 Ramanathan R. Stakhovsky S.
22692 Troicki V. Jung J.
22693 Eubanks C. Johnson S.
22694 Zverev M. Smyczek T.
22695 Bagnis F. Klizan M.
22696 Lorenzi P. Torebko P.
22697 Travaglia S. Fabbiano T.
22698 Serdarusic N. Trungelliti M.
22699 Balazs A. Galovic V.
22700 Rublev A. Haase R.
22701 Mayer L. Andujar P.
22702 Krajinovic F. Daniel T.
22703 Caruso S. Moutet C.
22704 Chardy J. Garin C.
22705 Jarry N. Ymer M.
22706 Ramos-Vinolas A. Verdasco F.
22707 Carballes Baena R. Dzumhur D.
22708 Bublik A. Troicki V.
22709 Ebden M. Kudla D.
22710 Granollers M. Thompson J.
22711 Ivashka I. Eubanks C.
22712 Sandgren T. Mannarino A.
22713 Zverev M. Andreozzi G.
22714 Isner J. Majchrzak K.
22715 Humbert U. Ramanathan R.
22716 Balazs A. Krajinovic F.
22717 Mayer L. Vesely J.
22718 Travaglia S. Fognini F.
22719 Djere L. Lorenzi P.
22720 Delbonis F. Cuevas P.
22721 Sousa J. Ymer E.
22722 Gasquet R. Novak D.
22723 Londero J.I. Dellien H.
22724 Bagnis F. Serdarusic N.
22725 Lajovic D. Rublev A.
22726 Ca

22978 Khachanov K. Auger-Aliassime F.
22979 Fognini F. Mannarino A.
22980 Nadal R. Pella G.
22981 Bautista Agut R. Gasquet R.
22982 Monfils G. Hurkacz H.
22983 Medvedev D. Thiem D.
22984 Khachanov K. Zverev A.
22985 Nadal R. Fognini F.
22986 Monfils G. Bautista Agut R.
22987 Medvedev D. Khachanov K.
22988 Nadal R. Monfils G.
22989 Nadal R. Medvedev D.
22990 De Minaur A. Cecchinato M.
22991 Isner J. Lajovic D.
22992 Albot R. Cilic M.
22993 Mannarino A. Garin C.
22994 Pouille L. Kudla D.
22995 Nishioka Y. Thompson J.
22996 Kecmanovic M. Auger-Aliassime F.
22997 Querrey S. Herbert P.H.
22998 Pella G. Ruud C.
22999 Londero J.I. Berrettini M.
23000 Gasquet R. Murray A.
23001 Struff J.L. Karlovic I.
23002 Carreno Busta P. Simon G.
23003 Kyrgios N. Sonego L.
23004 Wawrinka S. Dimitrov G.
23005 Goffin D. Fritz T.
23006 Rublev A. Basilashvili N.
23007 Paire B. Verdasco F.
23008 Tiafoe F. Monfils G.
23009 Schwartzman D. Djere L.
23010 Shapovalov D. Sousa J.
23011 Medvedev D. Edmund K.
23012 Baut

23263 Coric B. Ruud C.
23264 Sousa J. Kukushkin M.
23265 Gerasimov E. Berrettini M.
23266 Medvedev D. Rublev A.
23267 Bedene A. Paire B.
23268 Tsonga J.W. Pouille L.
23269 Coric B. Sousa J.
23270 Medvedev D. Gerasimov E.
23271 Tsonga J.W. Bedene A.
23272 Medvedev D. Coric B.
23273 Garin C. Edmund K.
23274 Klahn B. Jung J.
23275 Evans D. Bai Y.
23276 Norrie C. Gojowczyk P.
23277 Ramos-Vinolas A. Nishioka Y.
23278 Wu D. Ito T.
23279 Mannarino A. Zhang Ze.
23280 Carreno Busta P. Albot R.
23281 Lajovic D. Londero J.I.
23282 Verdasco F. Popyrin A.
23283 Harris L. Fucsovics M.
23284 Gerasimov E. Li Z.
23285 Andujar P. Johnson S.
23286 Kecmanovic M. Ruud C.
23287 Zhang Zh. Koepfer D.
23288 Dzumhur D. Cecchinato M.
23289 De Minaur A. Millman J.
23290 Murray A. Sandgren T.
23291 Sousa J. Chung H.
23292 Thompson J. Pospisil V.
23293 Bublik A. Fritz T.
23294 Shapovalov D. Berankis R.
23295 Kwon S.W. Pouille L.
23296 Seppi A. Kyrgios N.
23297 Ramos-Vinolas A. Kecmanovic M.
23298 Monfils G. Norrie 

23552 Federer R. Albot R.
23553 De Minaur A. Fritz T.
23554 Struff J.L. Laaksonen H.
23555 Carreno Busta P. Shapovalov D.
23556 Querrey S. Mannarino A.
23557 Rublev A. Bublik A.
23558 Kukushkin M. Coric B.
23559 Monfils G. Novak D.
23560 Berrettini M. Dimitrov G.
23561 Khachanov K. Fucsovics M.
23562 Opelka R. Goffin D.
23563 Bautista Agut R. Gasquet R.
23564 Tsitsipas S. Berankis R.
23565 Wawrinka S. Tiafoe F.
23566 Krajinovic F. Fognini F.
23567 Schwartzman D. Querrey S.
23568 Carreno Busta P. Kukushkin M.
23569 Rublev A. Chung H.
23570 Bedene A. Simon G.
23571 Thiem D. Verdasco F.
23572 Monfils G. Sinner J.
23573 Opelka R. Bautista Agut R.
23574 De Minaur A. Struff J.L.
23575 Federer R. Wawrinka S.
23576 Tsitsipas S. Krajinovic F.
23577 Berrettini M. Rublev A.
23578 Thiem D. Carreno Busta P.
23579 Schwartzman D. Khachanov K.
23580 Monfils G. Bedene A.
23581 De Minaur A. Opelka R.
23582 Federer R. Tsitsipas S.
23583 Thiem D. Berrettini M.
23584 Schwartzman D. Monfils G.
23585 Federer